线性回归整体实现步骤：

1. 生成数据集
2. 用dataloader读取数据，同时打乱顺序
3. 用iter构造python迭代器，用next从迭代器中获取第一项（这里也可以使用batch进行批量读取）
4. 定义神经网络
5. 定义损失函数
6. 定义优化算法
7. 训练，包括前向传播、反向传播、更新参数三步骤
8. 输出误差

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [2]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [3]:
next(iter(data_iter))

[tensor([[ 0.4272,  1.8039],
         [-0.3670,  2.2096],
         [-0.4712,  1.2598],
         [-0.4849, -0.7443],
         [-0.2421,  0.5400],
         [ 0.8013,  1.2085],
         [ 0.4275,  0.3910],
         [-1.6609, -1.3828],
         [ 0.1395, -0.3381],
         [-0.3693, -0.8970]]),
 tensor([[-1.0602],
         [-4.0561],
         [-1.0326],
         [ 5.7546],
         [ 1.8812],
         [ 1.7134],
         [ 3.7228],
         [ 5.5748],
         [ 5.6357],
         [ 6.5028]])]

In [4]:
# nn是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

In [5]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [6]:
loss = nn.MSELoss()

In [7]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [8]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X) ,y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000411
epoch 2, loss 0.000097
epoch 3, loss 0.000096


In [9]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([-0.0003, -0.0003])
b的估计误差： tensor([-0.0007])
